In [36]:
%matplotlib inline

from matplotlib import pyplot
import pandas
from pony import orm
from IPython import display
import jinja2
import db
import seaborn
import os
import warnings
import pathlib

In [ ]:
os.chdir('data')
db.use_db(str(pathlib.Path.cwd() / 'course.sqlite'))

In [ ]:
exam_id = os.environ['EXAM_ID']

full_scores = db.full_exam_data(exam_id)
problem_scores = full_scores.iloc[:, full_scores.columns.get_level_values(1) == 'total']
problem_scores.columns = problem_scores.columns.get_level_values(0)

with orm.db_session:
    display.display_html(display.HTML('<h1>Statistics for {}<h1>'.format(db.Exam[exam_id].name)))

warnings.simplefilter('ignore')

## Summary statistics

In [ ]:
problem_scores.fillna(0).astype(int).describe().round(2)

## Point distributions

In [ ]:
for column in problem_scores.columns:
    fig = pyplot.figure()
    ax = fig.add_subplot(1, 1, 1)
    seaborn.distplot(problem_scores[column], kde=False, ax=ax)
    pyplot.show()
    pyplot.close(fig)

## Correlations between different problems.

In [ ]:
problem_scores.fillna(0).corr().round(2)

## Lenghty description of all possible feedback options and how often they were received

In [ ]:
feedback_template = jinja2.Template("""<ul>
{% for fo in feedback_options %}
<li> {{ fo.text }}: {{ fo.description }} </li>
{% endfor %}
</ul>
""")

stats = ''

for problem in db.Exam[exam_id].problems.order_by(lambda p: p.name):
    if not orm.max(problem.feedback_options.score, default=0):
        continue
    df = pandas.DataFrame({fo.text: (fo.solutions.count(), fo.score) 
                           for fo in problem.feedback_options if fo.solutions.count()}, 
                          index=['amount', 'score']).T.fillna(0).astype(int)
    df.index.name = "Feedback"
    stats += '<h3>' + problem.name + '</h3>'
    stats += '<h4>Feedback frequencies and scores</h4>'
    stats += df._repr_html_()
    stats += '<hr><h4>Descriptions</h4>'
    stats += feedback_template.render(feedback_options=(fo for fo in problem.feedback_options
                                                        if fo.solutions.count()))


stats = display.HTML(stats)
stats